<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/GAN/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DCGAN
----
DCGAN 은 초기 심층 CNN 을 이용해 GAN 을 성공적으로 구현한 사례중 하나로, 다음과 같은 설계 원칙을 가지고 있다.

- MaxPooling2D 나 UpSampling2D 대신 stride>1 인 합성곱을 사용한다. stride 가 >1 일 경우, CNN 은 특징 맵 크기를 조정하는 방법을 학습하게 된다.

- Dense 계층의 사용을 피한다. 모든 계층에서 CNN 을 사용하도록 하며, Dense 계층은 생성기의 첫 번째 계층에서만 Z-vector 를 받기위해 사용된다. Dense 계층의 출력 크기는 조정되어 뒤따라 나오는 CNN 계층의 입력이 된다.

- 각 계층의 입력이 평균 0 에 단위 분산을 가져서 학습을 안정화 시키도록 Batch Normaliztion 을 사용한다. 생성기 출력 계층과 판별기 입력 계층에는 Batch Normalization을 사용하지 않는다.

- Rectitfied Linear Unit, ReLU 는 tanh 활성화 함수를 사용하는 출력 계층을 제외하고 생성기 전 층에서 사용된다. 아래에서는 tanh 대신 sigmoid 가 사용되는데, 일반적으로 MNIST 숫자에 대해 더 안정적으로 훈련시킬 수 있기 때문이다.

- 판별기의 전 계층에서 Leaky ReLU 를 사용한다. 

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model

import numpy as np
import math
import matplotlib.pyplot as plt
import os

모델 Build 를 위한 함수
-----

## 생성자 함수구성

In [2]:
# 생성기 모델 구성
"""
가짜 이미지 생성을 위한 BN-ReLU-Conv2DTranspose 스택 
출력계층의 활성화 함수로는 sigmoid 를 사용한다.

함수의 인수
inputs(Layer) : 생성기의 입력계층(z-vector 형태)
image_size: 이미지의 한 축의 목표 scale  (정사각형으로 가정)

return 
model: 생성기 모델
"""

def build_generator(inputs, image_size):
  image_resize = image_size//4

  kernel_size = 5
  layer_filters = [128, 64, 32, 1]

  x = Dense(image_resize*image_resize*layer_filters[0])(inputs)
  x = Reshape((image_resize, image_resize, layer_filters[0]))(x)

  for filters in layer_filters:
    # 첫 두 합성곱 계층만 strides = 2 사용
    if filters > layer_filters[-2]:
      strides = 2
    else:
      strides = 1
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(filters=filters,
                        kernel_size = kernel_size,
                        strides = strides,
                        padding = 'same')(x)
  
  x = Activation('sigmoid')(x)

  generator = Model(inputs, x, name = 'generator')
  
  return generator

## 판별자 함수 구성

In [3]:
# 판별기 모델 구성 
"""
진짜와 가짜를 구분하는 LeakyReLu - Conv2D 스택으로 구성이 되어있다.
BN 으로는 네트워크가 수렴하지 않으므로 BN을 사용하지 않았다.

inputs(Layer): 판별기의 입력 계층 (이미지)

return
Model: 판별기 모델 
"""

def build_discriminator(inputs):
  kernel_size = 5
  layer_filters = [32, 64, 128, 256]

  x = inputs
  for filters in layer_filters:
    # 첫 3개의 합성곱 계층은 strides = 2 개 사용, 마지막 계층은 stride = 1 사용
    if filters == layer_filters[-1]:
      strides = 1
    else:
      strides = 2
    
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(filters= filters,
               kernel_size= kernel_size,
               strides= strides,
               padding= 'same')(x)
    
  x = Flatten()(x)
  x = Dense(1)(x)
  x = Activation('sigmoid')(x)

  discriminator = Model(inputs, x, name='discriminator')
  
  return discriminator

In [4]:
def plot_images(generator,
                noise_input,
                show=False,
                step=0,
                model_name="gan"):
    """
    생성한 이미지를 visualize 해주는 코드 
    """
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict(noise_input)
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

# 모델훈련

## 데이터셋 로딩

In [5]:
#MNIST 데이터 셋 로딩
(x_train,_), (x_test, _) = mnist.load_data()

# CNN 을 위한 데이터형상 조정 및 정규화
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32')/255

## 모델 훈련을 위한 네트워크 구성

In [6]:
model_name = "dcgan_mnist"

# 네트워크 매개 변수 지정
latent_size = 100
batch_size = 64
train_steps = 40000
lr = 2e-4
decay = 6e-8
input_shape = (image_size, image_size, 1)

In [7]:
# 판별기 모델 구성
inputs = Input(shape = input_shape, name = 'discriminator_input')
discriminator = build_discriminator(inputs)
optimizer = RMSprop(lr=lr, decay=decay)

discriminator.compile(loss = 'binary_crossentropy',
                      optimizer = optimizer,
                      metrics=['accuracy'])

discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discriminator_input (InputLa [(None, 28, 28, 1)]       0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 32)        832       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          51264     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 128)       

In [8]:
# 생성기 모델 구성
input_gen = (latent_size,)
inputs = Input(shape = input_gen, name = 'g_input')
generator = build_generator(inputs, image_size)

generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_input (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 128)         512       
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       409728    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       51

In [9]:
# 적대적 모델 구성 결합
optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
#적대적 모델을 훈련하는 동안 판별기의 가중치는 고정한다.
discriminator.trainable = False

#adversarial = generator + discriminator
adversarial = Model(inputs, discriminator(generator(inputs)), name=model_name)

adversarial.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

adversarial.summary()

Model: "dcgan_mnist"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_input (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
generator (Functional)       (None, 28, 28, 1)         1301505   
_________________________________________________________________
discriminator (Functional)   (None, 1)                 1080577   
Total params: 2,382,082
Trainable params: 1,300,801
Non-trainable params: 1,081,281
_________________________________________________________________


In [10]:
models = (generator, discriminator, adversarial)
params = (batch_size, latent_size, train_steps, model_name)

# 모델 훈련 함수 지정

GAN 의경우 맞춤 훈련을 위해 일반적인 fit() 함수를 사용하지 않는다. 그 대신, 데이터 배치가 주어졌을 때에 대해 단일 경사 업데이트를 실행하기 위해 train_on_batch() 함수를 사용한다.
아래의 train 함수는 위에서 구현한 모델을 훈련하기 위한 함수를 나타낸다.

In [11]:
def train(models, x_train, params):
  # 함수의 인수로는 앞선 셀에서 리스트로 지정된 models 와 params, 그리고 훈련 이미지인 x_train 이 있다.
  
  """
  판별기와 적대적 네트워크를 훈련한 후, 배치 단위로 판별기와 적대적 네트워크를 교대로 훈련한다. 
  우선 판별기는 제대로 레이블이 붙은 진짜와 가짜 이미지를 가지고 훈련 시킨 후, 
  다음으로 적대적 네트워크를 진짜인척 하는 가짜 이미지를 사용하여 훈련시킨다.
  """

  # GAN 모델 불러오기
  generator, discriminator, adversarial = models

  # 네트워크 매개변수
  batch_size, latent_size, train_steps, model_name

  # 500 단계 마다 생성기 이미지가 저장되도록 설정
  save_interval = 500

  #훈련 기간 동안 생성기 출력 이미지가 어떻게 변화하는지 보여주기 위한 노이즈 벡터
  noise_input = np.random.uniform(-1.0, 1.0, size = [16, latent_size])
  
  # 훈련 이미지의 개수
  train_size = x_train.shape[0]

  for i in range(train_steps):
    # 1 배치에 대해 판별기 훈련
    # 데이터 셋에서 임의로 진짜 이미지를 선택한다
    rand_indices = np.random.randint(0, train_size, size = batch_size)
    real_images = x_train[rand_indices]

    #생성기를 사용해 노이즈로 부터 가짜 이미지를 생성한다.
    
    # 노이즈 분포 사용해 노이즈 생성
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])

    # 가짜 이미지 생성
    fake_images = generator.predict(noise)
    
    # 진짜 이미지와 가짜이미지의 훈련데이터의 배치 
    x = np.concatenate((real_images, fake_images))
    # 레이블을 붙임
    y = np.ones([2*batch_size, 1])
    y[batch_size:, :] = 0.0

    #판별기 훈련 및 손실과 정확도 기록
    loss, acc = discriminator.train_on_batch(x, y)
    log = "%d:[discriminator loss = %f, acc: %f]" %(i, loss, acc)

    # 1 배치에 대한 적대적 네트워크 훈련 
    # label = 1.0 인 가짜 이미지로 구성된 배치 

    #판별기의 가중치가 고정되므로 생성기만 훈련된다.
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
    
    # 가짜 이미지에 진짜 혹은 1.0 으로 레이블
    y = np.ones([batch_size, 1])

    #판별기를 훈련시키는 것과 달리 변수에 가짜 이미지를 저장하지 않는다.
    #가짜 이미지는 분류를 위해 적대적 네트워크의 판별기 입력으로 전달됨

    loss, acc = adversarial.train_on_batch(noise, y)
    log = "%d:[adversarial loss = %f, acc: %f]" %(log, loss, acc)
    print(log)

    print(f'{i/train_steps} % completed')

    if (i+1) % save_interval == 0:
      if (i+1) == train_steps:
        show = True
      else:
        show = False

      plot_images(generator, noise_input=noise_input, show=show, step=(i+1), model_name=model_name)
    
  genrator.save(model_name+".h5")

In [ ]:
train(models, x_train, params)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1370:[adversarial loss = 0.737635, acc: 0.562500]
1371:[discriminator loss = 0.618724, acc: 0.656250]
1371:[adversarial loss = 1.811703, acc: 0.015625]
1372:[discriminator loss = 0.494859, acc: 0.757812]
1372:[adversarial loss = 1.028951, acc: 0.234375]
1373:[discriminator loss = 0.488608, acc: 0.773438]
1373:[adversarial loss = 1.437969, acc: 0.156250]
1374:[discriminator loss = 0.419216, acc: 0.843750]
1374:[adversarial loss = 1.015907, acc: 0.359375]
1375:[discriminator loss = 0.487256, acc: 0.750000]
1375:[adversarial loss = 1.451218, acc: 0.078125]
1376:[discriminator loss = 0.472718, acc: 0.804688]
1376:[adversarial loss = 0.973273, acc: 0.328125]
1377:[discriminator loss = 0.469631, acc: 0.796875]
1377:[adversarial loss = 1.775194, acc: 0.015625]
1378:[discriminator loss = 0.525845, acc: 0.710938]
1378:[adversarial loss = 0.932527, acc: 0.296875]
1379:[discriminator loss = 0.538183, acc: 0.671875]
1379:[adversarial loss = 1.755226, acc: 0.0625